In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
codebase = '/output/HiKER-SGG/'
sys.path.append("/output/HiKER-SGG/")
# sys.path.append('../../../')
# sys.path.append('../../../apex')

In [2]:
import torch
torch.__version__

'1.12.0+cu113'

In [3]:
exp_name = 'hikersgg_predcls_test' # Change to the experiment name

In [4]:
import os
from time import time as time_time
import numpy as np
# from torch import optim
from apex import amp
import torch
import pandas as pd
from tqdm import tqdm
write = tqdm.write

from torch.optim.lr_scheduler import ReduceLROnPlateau

from config import ModelConfig, BOX_SCALE, IM_SCALE
from torch.nn import functional as F
from lib.pytorch_misc import optimistic_restore, de_chunkize, clip_grad_norm
from lib.evaluation.sg_eval import BasicSceneGraphEvaluator, calculate_mR_from_evaluator_list, eval_entry
from lib.pytorch_misc import print_para
from dataloaders.visual_genome import VGDataLoader, VG

from lib.my_model_24 import KERN

# sg val
# import numpy
# import pyximport
# pyximport.install(setup_args={"script_args":["--compiler=mingw32"],
#                               "include_dirs":numpy.get_include()},
#                   reload_support=True)
# then delete "script_args":["--compiler=mingw32"],


In [5]:
# Change ckpt path for the evaluated model
# If you want to test on VG-C benchmark, include "-test_n" in the command line
conf = ModelConfig(f'''
-m predcls -p 2500 -clip 5
-ckpt ../data/checkpoints/kern_predcls/hikersgg_predcls_train/vgrel-10.tar
-b 3
-ngpu 1
-test
-lr 1e-4
''')

# Also load the corresponding confusion matrix
conf_matrix = np.load('/output/data/misc/conf/conf_mat_updated_8.npy') # Remember to change to the path of the confusion matrix
np.save('/output/data/misc/conf_mat_updated.npy', conf_matrix)

~~~~~~~~ Hyperparameters used: ~~~~~~~
ckpt : ../data/checkpoints/kern_predcls/hikersgg_predcls_train/vgrel-10.tar
save_dir : 
num_gpus : 1
num_workers : 8
lr : 0.0001
batch_size : 3
val_size : 5000
l2 : 0.0001
adamwd : 0.0
clip : 5.0
print_interval : 2500
mode : predcls
cache : 
adam : False
test : True
test_n : False
num_epochs : 50
use_resnet : False
use_proposals : False
pooling_dim : 4096
use_ggnn_obj : False
ggnn_obj_time_step_num : 3
ggnn_obj_hidden_dim : 512
ggnn_obj_output_dim : 512
use_obj_knowledge : False
obj_knowledge : 
use_ggnn_rel : False
ggnn_rel_time_step_num : 3
ggnn_rel_hidden_dim : 512
ggnn_rel_output_dim : 512
use_rel_knowledge : False
rel_knowledge : 
tb_log_dir : 
save_rel_recall : 


In [6]:
# modified
conf.MODEL.CONF_MAT_FREQ_TRAIN = '/output/data/misc/conf_mat_freq_train.npy'
conf.MODEL.LRGA.USE_LRGA = False
conf.MODEL.USE_ONTOLOGICAL_ADJUSTMENT = False
conf.MODEL.NORMALIZE_EOA = False
conf.num_workers = 9
# conf.MODEL.LRGA.K = 50
# conf.MODEL.LRGA.DROPOUT = 0.5
# conf.MODEL.GN.NUM_GROUPS = 1024//8

In [7]:
os.getcwd()

'/output/HiKER-SGG/ipynb/eval_predcls'

In [8]:
train, val, test = VG.splits(num_val_im=conf.val_size, filter_duplicate_rels=True,
                            use_proposals=conf.use_proposals,
                            filter_non_overlap=conf.mode == 'sgdet',
                            with_clean_classifier=False,
                            get_state=False, test_n=conf.test_n)


Dataloader NOT using BPL
Dataloader NOT using BPL
Dataloader NOT using BPL


In [9]:
ind_to_predicates = train.ind_to_predicates # ind_to_predicates[0] means no relationship
# Evaluate on test set
# Here we let val = test since we want to call val_epoch() for evaluation
if conf.test or conf.test_n:
    val = test

In [10]:
_, train_loader = VGDataLoader.splits(train, train, mode='rel',
                                               batch_size=conf.batch_size,
                                               num_workers=conf.num_workers,
                                               num_gpus=conf.num_gpus,
                                               pin_memory=True)

In [11]:
_, val_loader = VGDataLoader.splits(train, val, mode='rel',
                                               batch_size=conf.batch_size,
                                               num_workers=conf.num_workers,
                                               num_gpus=conf.num_gpus,
                                               pin_memory=True)

In [12]:
detector = KERN(classes=train.ind_to_classes, rel_classes=train.ind_to_predicates,
                num_gpus=conf.num_gpus, mode=conf.mode, require_overlap_det=True,
                use_resnet=conf.use_resnet, use_proposals=conf.use_proposals, pooling_dim=conf.pooling_dim,
                ggnn_rel_time_step_num=3, ggnn_rel_hidden_dim=1024, ggnn_rel_output_dim=None,
                graph_path=os.path.join(codebase, 'graphs/005/all_edges_with_sccluster2_pred_ent.pkl'),
                emb_path=os.path.join(codebase, 'graphs/001/emb_mtx_with_sccluster2_pred_ent.pkl'),
                rel_counts_path=os.path.join(codebase, 'graphs/001/pred_counts.pkl'),
                use_knowledge=True, use_embedding=True, refine_obj_cls=False,
                class_volume=1.0, with_clean_classifier=True, with_transfer=True, sa=True, config=conf,
               )

my_ggnn_10: not using use_ontological_adjustment. self.use_ontological_adjustment=False
!!!!!!!!!With Confusion Matrix Channel!!!!!
SA: Used adj_normalize


In [13]:
ckpt = torch.load(conf.ckpt)
optimistic_restore(detector, ckpt['state_dict'], skip_clean=False)
detector.cuda();

In [14]:
print(print_para(detector), flush=True)


 455.9M total parameters 
 ----- 
 
detector.roi_fmap.0.weight                        : [4096,25088]    (102760448) (grad)
roi_fmap.1.0.weight                               : [4096,25088]    (102760448) (grad)
roi_fmap_obj.0.weight                             : [4096,25088]    (102760448) (grad)
detector.roi_fmap.3.weight                        : [4096,4096]     (16777216) (grad)
roi_fmap.1.3.weight                               : [4096,4096]     (16777216) (grad)
roi_fmap_obj.3.weight                             : [4096,4096]     (16777216) (grad)
ggnn_rel_reason.ggnn.fc_mp_receive_ont_ent.model.0.linear.weight: [3840,3840]     (14745600) (grad)
ggnn_rel_reason.ggnn.fc_mp_receive_ont_pred.model.0.linear.weight: [2560,2560]     ( 6553600) (grad)
ggnn_rel_reason.obj_proj.weight                   : [1024,4096]     ( 4194304) (grad)
ggnn_rel_reason.rel_proj.weight                   : [1024,4096]     ( 4194304) (grad)
ggnn_rel_reason.ggnn.fc_mp_receive_ont_ent.model.2.linear.weight: [1024

In [15]:
from torch import no_grad as torch_no_grad
from tqdm import tqdm

def val_epoch():
    detector.eval()
    evaluator_list = [] # for calculating recall of each relationship except no relationship
    evaluator_multiple_preds_list = []
    for index, name in enumerate(ind_to_predicates):
        if index == 0:
            continue
        evaluator_list.append((index, name, BasicSceneGraphEvaluator.all_modes()))
        evaluator_multiple_preds_list.append((index, name, BasicSceneGraphEvaluator.all_modes(multiple_preds=True)))
    evaluator = BasicSceneGraphEvaluator.all_modes() # for calculating recall
    evaluator_multiple_preds = BasicSceneGraphEvaluator.all_modes(multiple_preds=True)

    prog_bar = tqdm(enumerate(val_loader), total=int(len(val)/val_loader.batch_size))

    with torch_no_grad():
        for val_b, batch in prog_bar:
            val_batch(conf.num_gpus * val_b, batch, evaluator, evaluator_multiple_preds, evaluator_list, evaluator_multiple_preds_list)

    recall = evaluator[conf.mode].print_stats()
    recall_mp = evaluator_multiple_preds[conf.mode].print_stats()

    mean_recall = calculate_mR_from_evaluator_list(evaluator_list, conf.mode)
    mean_recall_mp = calculate_mR_from_evaluator_list(evaluator_multiple_preds_list, conf.mode, multiple_preds=True)

    detector.train()
    return recall, recall_mp, mean_recall, mean_recall_mp


In [16]:
from torch.cuda.amp import autocast

def val_batch(batch_num, b, evaluator, evaluator_multiple_preds, evaluator_list, evaluator_multiple_preds_list):
    with autocast():
        det_res = detector[b]
    if conf.num_gpus == 1:
        det_res = [det_res]

    for i, (boxes_i, objs_i, obj_scores_i, rels_i, pred_scores_i) in enumerate(det_res):
        gt_entry = {
            'gt_classes': val.gt_classes[batch_num + i].copy(),
            'gt_relations': val.relationships[batch_num + i].copy(),
            'gt_boxes': val.gt_boxes[batch_num + i].copy(),
        }
        assert np.all(objs_i[rels_i[:, 0]] > 0) and np.all(objs_i[rels_i[:, 1]] > 0)

        pred_entry = {
            'pred_boxes': boxes_i * BOX_SCALE/IM_SCALE,
            'pred_classes': objs_i,
            'pred_rel_inds': rels_i,
            'obj_scores': obj_scores_i,
            'rel_scores': pred_scores_i,  # hack for now.
        }

        eval_entry(conf.mode, gt_entry, pred_entry, evaluator, evaluator_multiple_preds, 
                   evaluator_list, evaluator_multiple_preds_list)



In [17]:
detector.eval()

recall, recall_mp, mean_recall, mean_recall_mp = val_epoch()

100%|██████████| 26446/26446 [26:40<00:00, 16.52it/s]

======================predcls  recall with constraint============================
R@20: 0.202644
R@50: 0.249882
R@100: 0.267186
======================predcls  recall without constraint============================
R@20: 0.307338
R@50: 0.492531
R@100: 0.644086


======================predcls  mean recall with constraint============================
mR@20:  0.3078203206168518
mR@50:  0.36648185583374454
mR@100:  0.38839326286825604




======================predcls  mean recall without constraint============================
mR@20:  0.3945585751297657
mR@50:  0.5447217917113147
mR@100:  0.6564887474256572
